In [27]:
import numpy as np
import pandas as pd

In [28]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [29]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA1_Tr_J/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [30]:
mineralogy = mineralogy.dropna()

In [31]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [32]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [33]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [34]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
19846,68.902419,0.436649,15.253316,0.894456,2.048878,0.063279,1.194222,2.454456,4.481631,4.270694
19847,68.895685,0.436928,15.254489,0.894444,2.049787,0.063305,1.195752,2.457807,4.481645,4.270159
19848,68.889270,0.437193,15.255604,0.894433,2.050653,0.063329,1.197210,2.461001,4.481658,4.269649
19849,68.883235,0.437442,15.256652,0.894422,2.051467,0.063352,1.198582,2.464008,4.481670,4.269170
20334,68.952628,0.434570,15.244511,0.894543,2.042086,0.063089,1.182839,2.429532,4.481516,4.274687


In [35]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
19846,68.902419,0.436649,15.253316,0.894456,2.048878,0.063279,1.194222,2.454456,4.481631,4.270694
19847,68.895685,0.436928,15.254489,0.894444,2.049787,0.063305,1.195752,2.457807,4.481645,4.270159
19848,68.889270,0.437193,15.255604,0.894433,2.050653,0.063329,1.197210,2.461001,4.481658,4.269649
19849,68.883235,0.437442,15.256652,0.894422,2.051467,0.063352,1.198582,2.464008,4.481670,4.269170
20334,68.952628,0.434570,15.244511,0.894543,2.042086,0.063089,1.182839,2.429532,4.481516,4.274687
...,...,...,...,...,...,...,...,...,...,...
208816,73.404988,0.243569,13.936151,0.884677,1.355695,0.043426,0.378526,0.722456,4.369148,4.661364
208817,73.419325,0.242961,13.929954,0.884576,1.353271,0.043355,0.376692,0.718735,4.368386,4.662746
208818,73.432466,0.242403,13.924261,0.884483,1.351049,0.043290,0.375016,0.715334,4.367685,4.664013
208819,73.444446,0.241895,13.919061,0.884398,1.349023,0.043230,0.373492,0.712243,4.367044,4.665168


In [37]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:90000]

In [38]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA1_Tr_J/area1_Tr_J_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA1_Tr_J/area1_Tr_J_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [41]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1_Tr_J/area1_Tr_J_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1_Tr_J/area1_Tr_J_2_results_excel.txt")

In [42]:
df_final = pd.concat([df_final1, df_final2])

In [44]:
df_final

,Q,P,K
SAMPLE,,,
19846,19.054792,43.018048,29.028022
19847,19.042517,43.022871,29.024907
19848,19.030882,43.027438,29.021900
19849,19.019936,43.031750,29.019072
20334,19.145943,42.981854,29.051565
...,...,...,...
208816,27.381741,36.855054,31.241367
208817,27.407507,36.830765,31.248939
208818,27.431187,36.808491,31.255787


## QAPF classification

In [45]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [46]:
df_final["QAPF"].unique()

array(['monzo granite', 'quartz monzonite'], dtype=object)

In [47]:
df_final.to_excel("../_CIPW/CIPW/AREA1_Tr_J/qapf.xlsx", index=True)

-------

In [48]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA1_Tr_J/QAPF_counts.xlsx")

-----

In [49]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area1_Tr_J.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [50]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control
QAPF_control.to_excel("../_CIPW/CIPW/AREA1_Tr_J/QAPF_control_points.xlsx", index=True)